In [17]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [18]:
# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
from src.models.EMOS_global.EMOS_global_load_score import *
from src.models.EMOS_local.EMOS_local_load_score import *
from src.models.EMOS_local.EMOS_local_load_model import *
from src.models.EMOS_global.EMOS_global_load_model import *
from src.models.EMOS_local.EMOS_local_hyper_tune import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
import data.processed.load_data_processed_denormed as ldpd
from src.models.CRPS_baseline.CRPS_load import *
from src.models.EMOS_local.EMOS_local_train import *

<IPython.core.display.Javascript object>

### 1. Load Scores

In [19]:
# Load EMOS local scores
EMOS_local_t2m_scores = EMOS_local_load_score("t2m")
# EMOS_local_ws10_scores = EMOS_local_load_score("ws10")

# Load EMOS global scores
EMOS_global_t2m_scores = EMOS_global_load_score("t2m")
EMOS_global_ws10_scores = EMOS_global_load_score("ws10")

# Load CRPS scores
CRPS_global_t2m_scores = crps_load_lead_lat_lon("t2m")
CRPS_global_ws10_scores = crps_load_lead_lat_lon("ws10")

<IPython.core.display.Javascript object>

In [22]:
len(EMOS_local_t2m_scores)

31

<IPython.core.display.Javascript object>

In [26]:
for lead_time in range(len(EMOS_local_t2m_scores)):
    print(
        f"EMOS_local: {EMOS_local_t2m_scores[lead_time].mean()}, EMOS_global: {EMOS_global_t2m_scores[lead_time].mean()}, CRPS: {CRPS_global_t2m_scores[lead_time].mean()}"
    )

EMOS_local: 0.5444882826810865, EMOS_global: 0.4673401873320892, CRPS: 0.5084816639136492
EMOS_local: 0.5053172433515578, EMOS_global: 0.5297212701322601, CRPS: 0.5347707002046389
EMOS_local: 0.5141073081029861, EMOS_global: 0.5105776602793739, CRPS: 0.5150757989800734
EMOS_local: 0.49568416761621475, EMOS_global: 0.5187042066593011, CRPS: 0.5301602687833538
EMOS_local: 0.5764168723142572, EMOS_global: 0.5379253302273839, CRPS: 0.5624034501418446
EMOS_local: 0.5760648372527605, EMOS_global: 0.5924092410007133, CRPS: 0.6306282599852645
EMOS_local: 0.5965537768976911, EMOS_global: 0.5920924175090567, CRPS: 0.6316119778923832
EMOS_local: 0.5809833473325556, EMOS_global: 0.6020434806248538, CRPS: 0.6454345388357919
EMOS_local: 0.6659074363281077, EMOS_global: 0.6251029754078069, CRPS: 0.6802850949781106
EMOS_local: 0.6791068504334168, EMOS_global: 0.6920910937109535, CRPS: 0.7561511514525252
EMOS_local: 0.7012070763260327, EMOS_global: 0.6984210795617619, CRPS: 0.7596827249402444
EMOS_loca

<IPython.core.display.Javascript object>

### 3. Load Hyperparameter tuning scores:

In [12]:
EMOS_local_ws10_hyper_scores = EMOS_local_load_hyper_score("ws10")

<IPython.core.display.Javascript object>

In [15]:
len(EMOS_local_ws10_hyper_scores)

2

<IPython.core.display.Javascript object>

In [16]:
EMOS_local_ws10_hyper_scores

[[(512, 30, 0.001, 'Adam'), 1.740081858749573],
 [(32, 30, 0.1, 'Adam'), 0.5986838517471765]]

<IPython.core.display.Javascript object>

### 3. Trying to load models:

In [ ]:
EMOS_local_t2m_0 = EMOS_local_load_model(2, 0)

In [24]:
lat = 2
lon = 2
lead_time = 0

<IPython.core.display.Javascript object>

In [28]:
test_var_denormed = ldpd.load_data_all_train_proc_denorm()[2]

<IPython.core.display.Javascript object>

In [29]:
X_test_var_denormed = test_var_denormed[
    list(test_var_denormed.data_vars.keys())[0]
].isel(lead_time=lead_time, lat=lat, lon=lon)
y_test_var_denormed = test_var_denormed[
    list(test_var_denormed.data_vars.keys())[1]
].isel(lead_time=lead_time, lat=lat, lon=lon)

<IPython.core.display.Javascript object>

In [30]:
EMOS_local_t2m_0[lat][lon].evaluate(
    [
        X_test_var_denormed.isel(mean_std=0).values.flatten(),
        X_test_var_denormed.isel(mean_std=1).values.flatten(),
    ],
    y_test_var_denormed.values.flatten(),
)

45/45 [==============================] - 0s 927us/step - loss: 402.3280


402.3279724121094

<IPython.core.display.Javascript object>